In [25]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint
from xml.etree.ElementTree import parse
from unidecode import unidecode

In [26]:
# Reference to Data Auditing process (project_submission_doc.pdf)
# 1) Provide a programmable reference to correct data
import normalized_stations

In [27]:
# Reference to Data Auditing process (project_submission_doc.pdf)
# 2) Sample the OSM file and clean station names
actual_stations = []
decoded_stations = {}


def parse_and_collect_nodes(osmfile):
    osm_file = open(osmfile, "r")
    doc = ET.iterparse(osm_file, events=("start","end"))
    try:
        for event, elem in doc:
            if event == "start" and elem.tag == "node":
                process_every_node(elem.iter("tag"))
            elif event == "end":
                continue
    except:
        print "End of file reached"

def process_every_node(itero):
    k_tags = []
    v_tags = []
    for tag in itero:
        k_tags.append(tag.attrib['k']) 
        v_tags.append(tag.attrib['v'])
    extract_stations(k_tags, v_tags)    

def extract_stations(k_tags, v_tags):
    if "subway" in k_tags:
        actual_stations.append(v_tags[0])

def to_ascii_actual_stations():
    for index, station in enumerate(actual_stations):
        # decoded_stations.append(unidecode(station))
        decoded_stations[index] = unidecode(station)

In [28]:
# Reference to Data Auditing process (project_submission_doc.pdf)
# 3) First pass stations and the rest and 4) Dealing with the “no match” stations names
match_stations   = {}
nomatch_stations = {}
dict_of_tokeners = {}
final_review_nomatch = []

tokens_to_ignore = [" de ", " los ", "-", " La ", " Los ", "los", "\(E\)", "\(C\)", "\(D\)"]
## split each station name in final_normalization in tokens ignoring the ones in tokens_to_ignore
pattern = '|'.join(tokens_to_ignore)

# irst pass of the actual_stations collected from OSM through a compare with official stations
def split_match_nomatch_stations():
    for key in decoded_stations.keys():
        if decoded_stations[key] in normalized_stations.all_stations():
            # print "found : {}, index: {}".format(decoded_stations[key],key)
            match_stations[key] = decoded_stations[key]
        else:
            nomatch_stations[key] = decoded_stations[key]

def split_nomatch():
    for key in nomatch_stations.keys():        
        tokeners = re.sub(pattern, ' ', nomatch_stations[key])
        dict_of_tokeners[key] = tokeners

def review_nomatch(tokener, key):
    nomatch_dict_key = key
    tokis = tokener.split()
    match_counter = defaultdict(int)
    for station in normalized_stations.all_stations():
        for tok in tokis:
            if tok in station:
                match_counter[station] += 1
                # print "nomatch_dict_key {} || station {} || complete {} || count {}".format(nomatch_dict_key,station,nomatch_stations[nomatch_dict_key], match_counter[station])
                # format(nomatch_dict_key,station,nomatch_stations[nomatch_dict_key], match_counter[station])
                datum = {"nomatch_key" : nomatch_dict_key, "norm_station" : station, "nomatch_station" : nomatch_stations[nomatch_dict_key], "count" : match_counter[station]}
                final_review_nomatch.append(datum)



In [29]:
OSMFILE = "bs_as_subway.osm"

In [ ]:
# Auditing process execution

In [30]:
parse_and_collect_nodes(OSMFILE)
# calls process_every_node()
# --> calls extract_stations

End of file reached


In [31]:
to_ascii_actual_stations()

In [32]:
split_match_nomatch_stations()

In [33]:
split_nomatch()

In [36]:
for key in dict_of_tokeners.keys():
    review_nomatch(dict_of_tokeners[key], key)

In [37]:
for dat in final_review_nomatch:
    print dat

{'count': 1, 'norm_station': 'Plaza de Mayo', 'nomatch_station': 'Plaza de los Virreyes - Eva Peron', 'nomatch_key': 1}
{'count': 1, 'norm_station': 'Plaza Miserere', 'nomatch_station': 'Plaza de los Virreyes - Eva Peron', 'nomatch_key': 1}
{'count': 1, 'norm_station': 'Plaza Italia', 'nomatch_station': 'Plaza de los Virreyes - Eva Peron', 'nomatch_key': 1}
{'count': 1, 'norm_station': 'Plaza de los Virreyes Eva Peron', 'nomatch_station': 'Plaza de los Virreyes - Eva Peron', 'nomatch_key': 1}
{'count': 2, 'norm_station': 'Plaza de los Virreyes Eva Peron', 'nomatch_station': 'Plaza de los Virreyes - Eva Peron', 'nomatch_key': 1}
{'count': 3, 'norm_station': 'Plaza de los Virreyes Eva Peron', 'nomatch_station': 'Plaza de los Virreyes - Eva Peron', 'nomatch_key': 1}
{'count': 4, 'norm_station': 'Plaza de los Virreyes Eva Peron', 'nomatch_station': 'Plaza de los Virreyes - Eva Peron', 'nomatch_key': 1}
{'count': 1, 'norm_station': 'Av. La Plata', 'nomatch_station': 'Avenida La Plata', 'nom